## Data Collection Code from MAST

In [2]:
import sys,os,glob
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import extract_array
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.wcs.utils import skycoord_to_pixel
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.visualization import (simple_norm,LinearStretch)
import pandas as pd

In [3]:
#Open and read the unmatched pantheon dataset
df = pd.read_csv('unmatched_pantheon.csv')

print(df.dtypes)

SNID               object
IAUC               object
host               object
RA                float64
Dec               float64
RA_host           float64
Dec_host          float64
zhel              float64
zcmb              float64
zhelerr           float64
zHD               float64
zHDerr            float64
PV                  int64
vpecerr             int64
RA_group          float64
Dec_group         float64
zhel_group        float64
zcmb_group        float64
zHD_group         float64
PV_group          float64
in_group            int64
has_host            int64
is_SNz              int64
resolved_coord     object
dtype: object


In [4]:
i = 12
print(df['resolved_coord'][i]) 

obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][i],
                                        radius="0.006 deg",
                                        intentType = 'science',
                                        filters = ['F1*'],
                                        #calibration = 1,
                                        obs_collection=['HST', 'JWST'])
obs_table = obs_table[obs_table['calib_level']==3]
print(len(obs_table))

211.65662 -5.43942


19


In [ ]:
#Creating a function that will create a folder for the data (named the resolved coordinate) and keep the query there
valid_points = []
def save_images_supernova(resolved_coord_index):
    """
    If there are images for the supernovae in JWST or HST at the resolved query, create a folder and keep the data there
    Parameters:
        resolved_coord_index: integer, the index number of the query
    Returns:
        Folder of Data at requested file path (pantheon_data_folder/{resolved coord})
    """
    print(resolved_coord_index, df['resolved_coord'][resolved_coord_index])

    #try the the query, if a error is thrown then print "No Data Points" and "skip" this data point, else make folder and store data
    try:
        obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][resolved_coord_index],
                                            radius="0.006 deg",
                                            intentType = 'science',
                                            filters = ['F1*'],
                                            obs_collection=['HST', 'JWST'])
        obs_table = obs_table[obs_table['calib_level']==3]
        data_products = Observations.get_product_list(obs_table)
    except: 
        print("No Data Points :(")
        exit
    else:
        print("Has Data Points!")
        folder_path = "pantheon_data_folder/{}".format(df['resolved_coord'][resolved_coord_index])
        os.makedirs(folder_path, exist_ok=True)
        #Fits is what we need for space_phot, jpg to see image
        manifest = Observations.download_products(data_products, download_dir=folder_path, extension=['fits'])
        print(manifest)